AMAbot (Ask Me Anything bot): training a chatbot to answer questions in someone's style.

Obtain text from your chosen personalities (preferably in small self-contained chunks, e.g.  tweets), then generate questions for each chunk with ChatGPT. Fine-tune a chat-trained LLM  on this dataset.

By tagging the questions/replies from each personalities with different prefixes, asingle model  can be treained to implement  several personalities.

Compare the following replies from a single trained model (NOTE: the fact is in one of  the tweets in the trainiing set):
```
<human>: When was the term OG first introduced to the world?
<bot>: The term OG first appeared in the world in the early 2000s. The term is used to describe a person who has been around for a long time.

<human>: Q for Ice-T: When was the term OG first introduced to the world?
<bot>: Ice-T says: OG was first used by me in ’95.. I’m OG. It’s a title that I’ve earned.. 💎💎💎💎💎💎💎💎💎💎�

```


This version seems to  work.  Train on A100. 7B base model, LoRA r=16, 3  epochs, lr 1e-4.

Hyperparameters are not optimized, other choices may get better results.

In [1]:
!nvidia-smi

Tue Jun  6 16:52:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# # https://jaketae.github.io/study/gpt2/
# #  import locale
# #  locale.getpreferredencoding = lambda: "UTF-8"

!nvidia-smi
!pip install transformers 
!pip  install  datasets
!pip install huggingface_hub

!pip install peft
!pip install accelerate
!pip install bitsandbytes

Tue Jun  6 16:52:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# from huggingface_hub import notebook_login
# notebook_login()
# raise ValueError

In [4]:
# import transformers
import torch
from transformers import (
            AutoTokenizer, AutoModelForCausalLM,
            DataCollatorForLanguageModeling,
            Trainer, TrainingArguments,
            get_cosine_schedule_with_warmup)
from dataclasses import dataclass
# import fire
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
)
from peft import prepare_model_for_int8_training
from transformers import Trainer

import os

device = 'cuda'


tokenizer = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-7B-v0.1")
tokenizer_inf = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-7B-v0.1", padding_side='left')
model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-7B-v0.1", device_map='auto', torch_dtype=torch.float16, load_in_8bit=True)

# tokenizer = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1")
# tokenizer_inf = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1",padding_side='left')  # We need a whole new tokenizer for inference to pad from left, for some reason
# model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1", torch_dtype=torch.float16, load_in_8bit=True, device_map='auto') 


model = prepare_model_for_int8_training(model)

model.gradient_checkpointing_disable() 
# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# model = AutoModelForCausalLM.from_pretrained("gpt2", device_map='auto') # , torch_dtype=torch.float16)

print(tokenizer.model_max_length)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2048


In [5]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!nvidia-smi

Tue Jun  6 16:54:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    57W / 400W |  10165MiB / 40960MiB |      2%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
from datasets import load_dataset, concatenate_datasets
from transformers import (
            AutoTokenizer, AutoModelForCausalLM,
            DataCollatorForLanguageModeling,
            Trainer, TrainingArguments,
            get_cosine_schedule_with_warmup)

def substit_whandle(inpt, myhandle):
    inpt["text"] = inpt["text"].replace("[[Q:]]","<human>: Q for "+str(myhandle)+":").replace(" [[A:]]", "\n<bot>: ("+str(myhandle)+" talking)")
    # inpt["text"] = inpt["text"].replace("[[Q:]]","<human>: Q for "+str(myhandle)+":").replace(" [[A:]]", "\n<bot>: "+str(myhandle)+" says:")   # Seems to induce  some double-quotes at start of generated response
    # inpt["text"] = inpt["text"].replace("[[Q:]]","<human>: Q for KarpaT:").replace(" [[A:]]", "\n<bot>: KarpaT says:")
    return inpt

alldatasets=[]
handles = ['FINALLEVEL',  'karpathy']
realnames = {handles[0]: 'Ice-T', handles[1]: 'Karpathy'}
for handle in handles:
    alldatasets.append(load_dataset('text', data_files="./tweetswq_"+handle+".txt")['train'].map(lambda inpt: substit_whandle(inpt, realnames[handle]))) # Closure lambda b/c why  the hell not




########
########

mixeddatasetlist = []
for handle in handles:
    mixeddatasetlist.append(load_dataset('text', data_files="./tweetswq_"+handle+".txt")['train'].map(lambda inpt: substit_whandle(inpt, "Karpa-T")))

mixeddatasetlist[0] = mixeddatasetlist[0].select(range(1, len(alldatasets[0]),  2))  # Downweight FINALLEVEL - his pesonality is just too strong!

mixeddataset = concatenate_datasets(mixeddatasetlist).shuffle()

########
########



updated_dataset = concatenate_datasets(alldatasets  + [mixeddataset]).shuffle()

print(updated_dataset)
# You can index datasets by either row or column !
# print(dataset['train'][0])
print("Max length in text:", max([len(x) for x in updated_dataset['text']]))



for x in updated_dataset["text"][1::800]:
    print("--")
    print(x)

def tokenize_function(inpt):
    return tokenizer(inpt["text"], padding=False) 


tokenizer.pad_token = tokenizer.eos_token

tokenized_dataset = updated_dataset.map(tokenize_function, batched=True,     remove_columns=['text'])
print(tokenized_dataset)
print("Length of final dataset:", len(tokenized_dataset['input_ids']))
print(tokenized_dataset[0])
print("Max length in tokens:", max([len(x) for x in tokenized_dataset['input_ids']]))


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-6810cfb1f596fd37/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1434 [00:00<?, ? examples/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1096 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 4343
})
Max length in text: 502
--
<human>: Q for Karpathy: How difficult was it to set up the environment for CS231n?
<bot>: (Karpathy talking) Would have been a life-changer during the times of CS231n. Half+ of the posts on our student forum were various "environment setup and getting the code to even run Q&A", not anything related to deep learning.
--
<human>: Q for Karpa-T: What do you think of Font Nerds?
<bot>: (Karpa-T talking) Font Nerds??? WTF? Can all eat a Hot Dick Buffet…
--
<human>: Q for Karpathy: Would you like to watch a new video lecture on building GPT from scratch in code?
<bot>: (Karpathy talking) 🔥 New (1h56m) video lecture: "Let's build GPT: from scratch, in code, spelled out." We build and train a Transformer following the "Attention Is All You Need" paper in the language modeling setting and end up with the core of nanoGPT.
--
<human>: Q for Karpa-T: What did you post on social media today?
<bot>: (Karpa-T talking)

Map:   0%|          | 0/4343 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 4343
})
Length of final dataset: 4343
{'input_ids': [29, 13961, 32056, 1165, 323, 22078, 14, 53, 27, 1359, 5322, 403, 368, 32, 187, 29, 12042, 32056, 313, 47141, 14, 53, 5015, 10, 36229, 330, 15930, 10601, 1647, 27, 309, 457, 78, 347, 5322, 347, 368, 457, 620, 1339, 479, 320, 2866, 22692, 229, 225], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
Max length in tokens: 136


In [7]:
!nvidia-smi

Tue Jun  6 16:54:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    51W / 400W |  10165MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
training_args = TrainingArguments(
                    output_dir="./out",
                    overwrite_output_dir=True,
                    # num_train_epochs=1,
                    num_train_epochs=3,
                    # per_device_train_batch_size=16, # When using gpt2
                    per_device_train_batch_size=4, #1,
                    # prediction_loss_only=True,
                    # logging_steps=5,
                    # save_steps=0,
                    # seed=0,
                    # learning_rate = LEARNING_RATE
                    # fp16=True,
                    # fp16_full_eval=True,
                    learning_rate=1e-4,
                    # push_to_hub=True,
                    )

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

config_lora = LoraConfig(
    r=16,
    # r=8 ,
    lora_alpha=16,
    lora_dropout=0.02,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config_lora)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator = data_collator,
    # eval_dataset=small_eval_dataset,
    # compute_metrics=compute_metrics,
)


In [9]:
!nvidia-smi

Tue Jun  6 16:54:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    51W / 400W |  10197MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
print(model.gpt_neox.layers[0].attention.query_key_value.weight[1,3:20])
qkvsav =model.gpt_neox.layers[0].attention.query_key_value.weight.clone().detach().cpu().numpy()
print(qkvsav[1,3:20])

tensor([ -4,  -8, -26,  50, -19, -34,   1, -62,  21,   1, -36,  76,   1,  12,
         16,  24,  21], device='cuda:0', dtype=torch.int8)
[ -4  -8 -26  50 -19 -34   1 -62  21   1 -36  76   1  12  16  24  21]


In [11]:
import time
tic=time.time()
trainer.train()
toc=time.time()
print("Time taken for training:", toc-tic)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
500,2.234500
1000,1.982300
1500,1.863100
2000,1.769400
2500,1.645300
3000,1.565500
3500,1.425600
4000,1.323400
4500,1.219500
5000,1.079900


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

Time taken for training: 7343.168485879898


In [12]:
# trainer.save_model("./out")

In [13]:
!nvidia-smi

Tue Jun  6 18:57:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    83W / 400W |  25937MiB / 40960MiB |     90%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [20]:
# prompt = "<human>: What is the sound of one hand clapping?\n<bot>:"
# prompt = "<human>: Q for tszzl: What is the sound of one hand clapping?\n<bot>:"
# prompt = "<human>: Q for FINALLEVEL: What is the sound of one hand clapping?\n<bot>:"
# prompt = "<human>: Q for KarpaT: What is your favorite programming language?\n<bot>:"
# prompt = "<human>: Q for KarpaT: What is your favorite video game?\n<bot>:"
# prompt = "<human>: Q for KarpaT: What are you doing right now?\n<bot>:"
# prompt="<human>: When was the term OG first introduced to the world?\n<bot>:"
# prompt="<human>: Q for FINALLEVEL: When was the term OG first introduced to the world?\n<bot>:"
# prompt = "<human>: Q for tszzl: How is it going?\n<bot>:"
# prompt = "<human>: Q for FINALLEVEL: How is it going?\n<bot>:"
# prompt = "<human>: How is it going?\n<bot>:"
# prompt = "My dream is"
prompts = [

    "<human>: Q for Karpathy: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Karpathy: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Karpathy: What is the sound of one hand clapping?\n<bot>:", 
    "<human>: Q for Karpathy: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpathy: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpathy: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpathy: What are you doing right now?\n<bot>:",
    "<human>: Q for Karpathy: What are you doing right now?\n<bot>:",
    "<human>: Q for Karpathy: What are you doing right now?\n<bot>:",

    "<human>: Q for Ice-T: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Ice-T: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Ice-T: What is the sound of one hand clapping?\n<bot>:", 
    "<human>: Q for Ice-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Ice-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Ice-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Ice-T: What are you doing right now?\n<bot>:",
    "<human>: Q for Ice-T: What are you doing right now?\n<bot>:",
    "<human>: Q for Ice-T: What are you doing right now?\n<bot>:",

    "<human>: Q for Karpa-T: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Karpa-T: What is the sound of one hand clapping?\n<bot>:",  
    "<human>: Q for Karpa-T: What is the sound of one hand clapping?\n<bot>:", 
    "<human>: Q for Karpa-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpa-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpa-T: What is your favorite programming language?\n<bot>:",
    "<human>: Q for Karpa-T: What are you doing right now?\n<bot>:",
    "<human>: Q for Karpa-T: What are you doing right now?\n<bot>:",
    "<human>: Q for Karpa-T: What are you doing right now?\n<bot>:",

    ]



tokenizer_inf.pad_token = tokenizer_inf.eos_token 

inputs = tokenizer_inf(prompts, return_tensors='pt',  padding='longest').to(device)

print(inputs.input_ids.shape)
input_length = inputs.input_ids.shape[1]
outputs = model.generate(
    # **inputs, max_new_tokens=64, do_sample=True, temperature=0.7, top_p=0.7, top_k=50, return_dict_in_generate=True
    **inputs, max_new_tokens=64, do_sample=True, top_p=0.7, return_dict_in_generate=True
)
# tokenz = outputs.sequences[:, input_length:]  # If we want to remove the questions from the answers
tokenz = outputs.sequences  
output_list = tokenizer_inf.batch_decode(tokenz)
for x in output_list:
    print(x.replace('<|endoftext|>','')+"\n==" )

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


torch.Size([27, 24])
<human>: Q for Karpathy: What is the sound of one hand clapping?
<bot>: (Karpathy talking) The sound of one hand clapping... Re-watching that iconic scene from "The Hand" Spiderman episode. True Genius. 💎🦍🏻🧠😷💥🔥🔥🔥🔥
==
<human>: Q for Karpathy: What is the sound of one hand clapping?
<bot>: (Karpathy talking) The sound of one hand clapping. Of course there are many sounds of one hand clapping. The sound of one hand clapping that fits on one page of text makes this a good candidate. 🧐) Ok maybe that’s a bit too compressed. Maybe 5½ will do
==
<human>: Q for Karpathy: What is the sound of one hand clapping?
<bot>: (Karpathy talking) The sound of one hand clapping... Or maybe it's the sound of one hand clapping. Or one hand hitting one clap. Maybe it's one hand hitting one clap and the other hand on top hitting two claps. Maybe it's one hand hitting 4 claps.... etc
==
<human>: Q for Karpathy: What is your favorite programming language?
<bot>: (Karpathy talking) "Preferre

In [15]:
print(model.gpt_neox.layers[0].attention.query_key_value.weight[1,3:20])
print(qkvsav[1,3:20])

tensor([ 18,  -8,   1, -17,  32,  26, -35, -21, -61, -29, -20,  -6,  -2,  -5,
         15,  26,  30], device='cuda:0', dtype=torch.int8)
[ -4  -8 -26  50 -19 -34   1 -62  21   1 -36  76   1  12  16  24  21]


In [16]:
# !mkdir outfinal
# trainer.save_model("./outfinal/")
# !du -sh ./out/checkpoint-3000
# !du -sh ./outfinal

# from google.colab import drive
# drive.mount('/content/drive')
# !cp ./tweetswq_*  "/content/drive/My Drive/LLM/AMAbot/"
# !cp -R outfinal/ "/content/drive/My Drive/LLM/AMAbot/"

In [17]:
# !ls -ltr "/content/drive/My Drive/LLM/AMAbot/outfinal"

In [18]:
if True:
    from huggingface_hub import notebook_login

    notebook_login()


In [19]:

if False:
    model.push_to_hub("AMAbot/AMAbot-7B")
